### data

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import gurobipy as gp
from gurobipy import GRB

R_flat = pd.read_csv("result/result_R.csv")

I_len = R_flat["generator"].nunique()
T_len = R_flat["time"].nunique() 
S_len = R_flat["scenario"].nunique()

R = R_flat.pivot_table(
    values="value",
    index="generator",
    columns=["time", "scenario"]
).to_numpy()
R = R.reshape(I_len, T_len, S_len)

I, T, S = R.shape

price_q = pd.read_csv(
    "/Users/jangseohyun/Documents/workspace/symply/DER/DATA_price.csv"
)
price_q["Time"] = pd.to_datetime(price_q["Time"], format="%Y-%m-%d %H:%M")

price_q["Hour"] = price_q["Time"].dt.floor("h")
price_h = price_q.groupby("Hour").mean(numeric_only=True)

price = price_h.iloc[: S * T]

P_DA = np.array(
    [sum(price["Price"].iloc[t + s * T] for s in range(S)) / S * 1.2 for t in range(T)]
)
P_RT = np.array([[price["Price"].iloc[t + s * T] for s in range(S)] for t in range(T)])
P_PN = np.array(
    [sum(price["Price"].iloc[t + s * T] for s in range(S)) / S * 2 for t in range(T)]
)

only_profit = pd.read_csv('result/result_only_profit.csv').values

### settlement model

In [28]:
def my_callback(model, where):
    if where == GRB.Callback.MIP:
        obj_best = model.cbGet(GRB.Callback.MIP_OBJBST)
        obj_bound = model.cbGet(GRB.Callback.MIP_OBJBND)

        # MIP Gap 계산
        if obj_best != 0:  # obj_best가 0이 아닌 경우에만 계산
            mip_gap = abs(obj_best - obj_bound) / abs(obj_best)
        else:
            mip_gap = float("inf")  # obj_best가 0인 경우 무한대로 설정

        print(f"obj_best: {obj_best}, obj_bound: {obj_bound}, MIP Gap: {mip_gap:.4f}")


set = gp.Model("Settlement")
# set.Params.OutputFlag = 0
# set.setParam('TimeLimit', 600)
# set.setParam('Presolve', 0)
# set.setParam("PoolSolutions", 15)
# set.setParam("PoolSearchMode", 2)
# set.setParam("PoolGap", 0.05)
# set.setParam("MIPGap", 1e-3)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M_y = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
d_max_H = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="d_max_H")
# d_min_H = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="d_min_H")

e_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
e_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
M_e = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_e = set.addVars(T, S, vtype=GRB.BINARY, name="z_e")

profit_increase = set.addVars(I, vtype=GRB.CONTINUOUS, lb=0, name="profit_increase")

set.update()

# obj. func.
obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s])
    for i in range(I) for t in range(T) for s in range(S)
)

set.setObjective(obj, GRB.MAXIMIZE)

In [29]:
# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= M_y * z_y[i, t, s])
            set.addConstr(y_minus[i, t, s] <= M_y * (1 - z_y[i, t, s]))

# rt 조건
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= R[i, t, s])

# sum e+, sum e- 결정 지어주기
for t in range(T):
    for s in range(S):
        set.addConstr(
            gp.quicksum(R[i, t, s] for i in range(I))
            - gp.quicksum(x[i, t] for i in range(I))
            == gp.quicksum(e_plus[i, t, s] for i in range(I))
            - gp.quicksum(e_minus[i, t, s] for i in range(I))
        )

# sum e+, sum e- 둘 중 하나는 0이어야 함
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= M_e * z_e[t, s])
        set.addConstr(
            gp.quicksum(e_minus[i, t, s] for i in range(I)) <= M_e * (1 - z_e[t, s])
        )

# e+, e- 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                e_plus[i, t, s]
                == y_plus[i, t, s]
                - gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                e_minus[i, t, s]
                == y_minus[i, t, s]
                - gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
            )

# sum e+ <= sum R
for t in range(T):
    for s in range(S):
        set.addConstr(
            gp.quicksum(e_plus[i, t, s] for i in range(I))
            <= gp.quicksum(R[i, t, s] for i in range(I))
        )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 본인이 주는 양은 자신의 y+을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )
            # 본인이 받는 양은 자신의 y-을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

# # 수익증가율 정의
# for i in range(I):
#     set.addConstr(profit_increase[i] == (
#         gp.quicksum(
#             P_DA[t] * x[i, t]
#             + 1 / S * gp.quicksum(P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s] for s in range(S))
#             for t in range(T)
#         ) - only_profit[i].item()
#     ) / only_profit[i].item())

# # 수익증가율이 사람마다 같아야함
# for i in range(I):
#     set.addConstr(
#         profit_increase[i] == gp.quicksum(profit_increase[j] for j in range(I)) / I
#     )
    
# d_max_H 정의
for t in range(T):
    for s in range(S):
        for i in range(I):
                set.addConstr(
                    d_max_H[t, s] >= gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
            )

# d_max 제한
for t in range(T):
    for s in range(S):
        set.addConstr(
            d_max_H[t, s] <= 0.5 * gp.quicksum(y_plus[i, t, s] for i in range(I))
        )


# # d_min 정의
# for t in range(T):
#     for s in range(S):
#         for i in range(I):
#             set.addGenConstrIndicator(
#                 z_y[i, t, s],
#                 1,
#                 d_min_H[t, s] <= gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
#             )

# # 시간별 거래량 차이를 제한
# for t in range(T):
#     for s in range(S):
#         set.addConstr(
#             d_max_H[t, s] - d_min_H[t, s] <= 10
#         )

In [30]:
set.optimize()

if set.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array(
        [[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_minus_vals = np.array(
        [[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    d_vals = np.array(
        [
            [[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)]
            for i in range(I)
        ]
    )
    d_max_H_vals = np.array([[[d_max_H[t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    # d_min_H_vals = np.array([[[d_min_H[t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    e_plus_vals = np.array(
        [[[e_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    e_minus_vals = np.array(
        [[[e_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    z_y_vals = np.array(
        [[[z_y[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    z_e_vals = np.array([[z_e[t, s].x for s in range(S)] for t in range(T)])
    profit_increase_vals = np.array([[profit_increase[i].x for i in range(I)]])
    print("\n- - - - - - - - - - - - - - - - - - - - - - - -")
    print("Optimal solution found!")
    print(set.objVal)
else:
    print("\nNo optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50400 rows, 73210 columns and 327840 nonzeros
Model fingerprint: 0x2c3a1dda
Variable types: 67930 continuous, 5280 integer (5280 binary)
Coefficient statistics:
  Matrix range     [5e-01, 9e+01]
  Objective range  [4e-04, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 9e+01]
Presolve removed 49483 rows and 71050 columns
Presolve time: 0.65s
Presolved: 917 rows, 2160 columns, 7280 nonzeros
Variable types: 1981 continuous, 179 integer (179 binary)

Root relaxation: objective 1.191030e+04, 802 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 11910.2972    0   20          - 11910.2972      -     -    0s
H    0   

In [31]:
if set.SolCount > 0:
    print(f"Best objective found: {set.ObjVal}")
    for v in set.getVars():
        print(f"{v.VarName} = {v.X}")
else:
    print("No feasible solution found.")

Best objective found: 11875.85223752153
x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[0,6] = 0.0
x[0,7] = 0.0
x[0,8] = 0.0
x[0,9] = 0.0
x[0,10] = 3.890357437238184
x[0,11] = 1.2928059861450052
x[0,12] = 1.4032075686147216
x[0,13] = 4.1841474923499415
x[0,14] = 0.9249543163117776
x[0,15] = 10.043085438966079
x[0,16] = 0.4067704609004643
x[0,17] = 0.0692330012461672
x[0,18] = 0.0
x[0,19] = 0.0
x[0,20] = 0.0
x[0,21] = 0.0
x[0,22] = 0.0
x[0,23] = 0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.0
x[1,6] = 0.0
x[1,7] = 0.0
x[1,8] = 0.0
x[1,9] = 0.0
x[1,10] = 3.5907231407575786
x[1,11] = 0.4959473357874331
x[1,12] = 4.617940711267469
x[1,13] = 8.20429347709188
x[1,14] = 2.8936295086383392
x[1,15] = 0.0
x[1,16] = 0.9471244937063892
x[1,17] = 0.0
x[1,18] = 0.0
x[1,19] = 0.0
x[1,20] = 0.0
x[1,21] = 0.0
x[1,22] = 0.0
x[1,23] = 0.0
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 0.0
x[2,5] = 0.0
x[2,6] = 0.0
x[2,7] = 0.

### solution pool

In [32]:
print(f"Status: {set.status}")
if set.status == GRB.OPTIMAL:
    print("모델이 Optimal입니다")

    # Solution pool 확인
    print("\nSolution Pool 정보:")
    print(f"찾은 해의 개수: {set.SolCount}")

    # Best solution의 objective value
    best_obj = set.ObjVal
    print(f"\nBest Solution의 Objective Value: {best_obj}")

    # 각 솔루션의 목적함수 값 출력 및 best solution 여부 확인
    solutions = []
    for i in range(set.SolCount):
        set.params.SolutionNumber = i
        solutions.append((i, set.PoolObjVal))
        print(f"\nSolution {i}:")
        print(f"Objective value: {set.PoolObjVal}")
        print(f"Objective bound: {set.PoolObjBound}")
    # Solution pool 파라미터 확인
    print("\nSolution Pool 파라미터:")
    print(f"PoolSolutions: {set.Params.PoolSolutions}")
    print(f"PoolSearchMode: {set.Params.PoolSearchMode}")
    print(f"PoolGap: {set.Params.PoolGap}")
elif set.status == GRB.UNBOUNDED:
    print("모델이 Unbounded입니다")
    # Unbounded ray 확인
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INFEASIBLE:
    print("모델이 Infeasible입니다")
    # IIS 계산
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INF_OR_UNBD:
    print("모델이 Infeasible 또는 Unbounded입니다")
    # Presolve를 끄고 다시 시도
    set.setParam("Presolve", 0)
    set.optimize()
elif set.status == GRB.TIME_LIMIT:
    print("시간 제한에 도달했습니다")
    if set.SolCount > 0:
        print(f"현재까지의 최선해: {set.objVal}")
        print(f"Optimality Gap: {set.MIPGap}")

Status: 2
모델이 Optimal입니다

Solution Pool 정보:
찾은 해의 개수: 2

Best Solution의 Objective Value: 11875.85223752153

Solution 0:
Objective value: 11875.85223752153
Objective bound: 11876.199946851708

Solution 1:
Objective value: 11856.565941234956
Objective bound: 11876.199946851708

Solution Pool 파라미터:
PoolSolutions: 10
PoolSearchMode: 0
PoolGap: inf


In [33]:
# 솔루션 풀에서 해 확인
solution_count = set.SolCount
print(f"총 {solution_count}개의 해가 저장되었습니다.")

for i in range(solution_count):
    set.setParam(GRB.Param.SolutionNumber, i)
    print(f"\n해 {i+1}: 목적 함수 값 = {set.PoolObjVal}")
    # x 변수 값 출력
    for t in range(14, 15):
        for i in range(1, 5):
            print(f"x[{i},{t}] = {x[i, t].Xn}")

총 2개의 해가 저장되었습니다.

해 1: 목적 함수 값 = 11875.85223752153
x[1,14] = 2.8936295086383392
x[2,14] = 5.670012548583053
x[3,14] = 4.5442714360194945
x[4,14] = 4.867546287552169

해 2: 목적 함수 값 = 11856.565941234956
x[1,14] = 2.8936295086383392
x[2,14] = 5.670012548583053
x[3,14] = 4.5442714360194945
x[4,14] = 4.867546287552169


### 결과 분석

#### 수익 분석

In [34]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i, t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * e_plus[i, t, s].x
            total_rt_profit_obj += 1 / S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * e_minus[i, t, s].x
            total_penalty_cost_obj += 1 / S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = (
    total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj
)

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

DA: 8561.60
RT: 3648.34
Penalty: 334.09
목적 함수 기반 총 이익 (_obj): 11875.85


In [35]:
for t in range(T):
    avg_e_plus = sum(e_plus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    avg_e_minus = sum(e_minus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    print(
        f"[시간 {t}] x: {sum(x_vals[i, t] for i in range(I)):.3f}, e+: {avg_e_plus:.3f}, e-: {avg_e_minus:.3f}"
    )

total_x = sum(x_vals[i, t] for i in range(I) for t in range(T))
print(f"총 하루 commitment: {total_x:.3f}")

[시간 0] x: 0.000, e+: 0.000, e-: 0.000
[시간 1] x: 0.000, e+: 0.000, e-: 0.000
[시간 2] x: 0.000, e+: 0.000, e-: 0.000
[시간 3] x: 0.000, e+: 0.000, e-: 0.000
[시간 4] x: 0.000, e+: 0.000, e-: 0.000
[시간 5] x: 0.000, e+: 0.000, e-: 0.000
[시간 6] x: 0.000, e+: 0.000, e-: 0.000
[시간 7] x: 0.000, e+: 0.000, e-: 0.000
[시간 8] x: 0.541, e+: 1.500, e-: 0.043
[시간 9] x: 7.714, e+: 5.072, e-: 0.318
[시간 10] x: 24.230, e+: 8.442, e-: 0.089
[시간 11] x: 28.485, e+: 16.456, e-: 1.217
[시간 12] x: 38.932, e+: 16.756, e-: 0.429
[시간 13] x: 39.040, e+: 18.203, e-: 0.250
[시간 14] x: 34.879, e+: 15.878, e-: 0.759
[시간 15] x: 25.752, e+: 9.758, e-: 0.873
[시간 16] x: 11.038, e+: 8.142, e-: 1.000
[시간 17] x: 0.757, e+: 4.032, e-: 0.034
[시간 18] x: 0.000, e+: 0.495, e-: 0.000
[시간 19] x: 0.000, e+: 0.000, e-: 0.000
[시간 20] x: 0.000, e+: 0.000, e-: 0.000
[시간 21] x: 0.000, e+: 0.000, e-: 0.000
[시간 22] x: 0.000, e+: 0.000, e-: 0.000
[시간 23] x: 0.000, e+: 0.000, e-: 0.000
총 하루 commitment: 211.369


### exchange process

In [36]:
for t in range(12,14):
    x_sum = sum(x_vals[i, t] for i in range(I))

    for s in range(S):
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))

        e_plus_sum = sum(e_plus_vals[i, t, s] for i in range(I))
        e_minus_sum = sum(e_minus_vals[i, t, s] for i in range(I))

        print(f"[t={t} s={s}]")

        print(f"x 합계: {x_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i,t]:.2f}", end=" ")
        print(")")

        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e+ 합계: {e_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e- 합계: {e_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print("거래 내역:")
        print(f"d_max: {d_max_H[t,s].x:.2f}")
        for i in range(I):
            for j in range(I):
                if d_vals[i, j, t, s] > 1e-6:
                    print(
                        f"발전기 {i}가 발전기 {j}에게 {d_vals[i,j,t,s]:.2f} 전력을 줌"
                    )
        print()

[t=12 s=0]
x 합계: 38.93 ([0] 1.40 [1] 4.62 [2] 4.39 [3] 6.30 [4] 5.38 [5] 4.64 [6] 0.76 [7] 4.11 [8] 4.47 [9] 2.87 )
y+ 합계: 29.40 ([0] 10.20 [1] 0.00 [2] 3.42 [3] 0.00 [4] 4.40 [5] 0.00 [6] 11.38 [7] 0.00 [8] 0.00 [9] 0.00 )
y- 합계: 14.62 ([0] 0.00 [1] 4.62 [2] 0.00 [3] 6.30 [4] 0.00 [5] 0.00 [6] 0.00 [7] 3.26 [8] 0.00 [9] 0.44 )
e+ 합계: 14.78 ([0] 10.20 [1] 0.00 [2] 3.42 [3] 0.00 [4] 1.16 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
e- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
거래 내역:
d_max: 14.70
발전기 4가 발전기 3에게 2.80 전력을 줌
발전기 4가 발전기 9에게 0.44 전력을 줌
발전기 6가 발전기 1에게 4.62 전력을 줌
발전기 6가 발전기 3에게 3.50 전력을 줌
발전기 6가 발전기 7에게 3.26 전력을 줌

[t=12 s=1]
x 합계: 38.93 ([0] 1.40 [1] 4.62 [2] 4.39 [3] 6.30 [4] 5.38 [5] 4.64 [6] 0.76 [7] 4.11 [8] 4.47 [9] 2.87 )
y+ 합계: 31.45 ([0] 0.00 [1] 5.02 [2] 8.37 [3] 0.00 [4] 1.23 [5] 0.00 [6] 8.12 [7] 4.73 [8] 0.00 [9] 3.97 )
y- 합계: 12.08 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 6.30 [4] 0.00 [5] 1.32 [6] 0.00 [7] 0.00

In [37]:
d_given = np.zeros(I)
d_received = np.zeros(I)

for i in range(I):
    d_given[i] = sum(
        sum(d_vals[i, j, t, s] for j in range(I) if j != i for t in range(T)) / S
        for s in range(S)
    )
    d_received[i] = sum(
        sum(d_vals[j, i, t, s] for j in range(I) if j != i for t in range(T)) / S
        for s in range(S)
    )

set_d = pd.DataFrame({"set_d_given": d_given, "set_d_received": d_received})
set_d.to_csv("result/result_set4_d.csv", index=False)

### 정산

In [38]:
total_der_profit = 0
der_profit = {}
for i in range(I):
    der_profit[i] = sum(P_DA[t] * x[i,t].x + sum(1/S * (P_RT[t,s] * e_plus[i,t,s].x - P_PN[t] * e_minus[i,t,s].x) for s in range(S)) for t in range(T))
    total_der_profit += der_profit[i]

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")

print("\n최종 profit:")
for i in range(I):
    print(f"[{i}] {der_profit[i]:.2f} ({(der_profit[i]-only_profit[i].item())/only_profit[i].item() * 100:.2f}%)")


모든 der의 profit 합계: 11875.85

최종 profit:
[0] 1347.07 (11.83%)
[1] 1049.45 (20.27%)
[2] 1680.39 (35.79%)
[3] 1255.81 (11.28%)
[4] 1285.93 (77.56%)
[5] 1023.84 (6.26%)
[6] 1496.06 (-13.66%)
[7] 1074.66 (-2.21%)
[8] 847.92 (10.27%)
[9] 814.73 (-12.46%)


### 결과 저장

In [39]:
set_profit_value = pd.DataFrame({'set_profit_value': der_profit})
set_profit_value.to_csv('result/result_set4_profit.csv', index=False)

set_obj = pd.DataFrame({'set_obj': [set.objVal]})
set_obj.to_csv('result/result_set4_obj.csv', index=False)